# Imports & Setup

In [ ]:
%load_ext autoreload
%autoreload 2
from utils import *
import utils.transformers.transformers as tr
import utils.transformers.sk4pandas as s4p

import os
import pickle
import urllib.request

import git
from IPython.core.magic import register_cell_magic
import mlflow
import pandas as pd
from sklearn import set_config
from sklearn.base import clone
from sklearn.compose import make_column_selector
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from scipy.stats import uniform, loguniform

git_repo = git.Repo(os.getcwd())

@register_cell_magic
def run_and_save(line, cell):
    'Run and save python code block to a file'
    with open(line, 'wt') as fd:
        fd.write(cell)
    code = compile(cell, line, 'exec')
    exec(code, globals())

# Init Datasets

In [ ]:
raw_main_df = pd.read_csv('data/train.csv')
raw_sub_df = pd.read_csv('data/test.csv')
raw_train_df, raw_test_df = train_test_split(raw_main_df, test_size=100, random_state=42)

X_all, y_all = raw_main_df.drop('Survived', axis=1), raw_main_df['Survived']
X_train, y_train = raw_train_df.drop('Survived', axis=1), raw_train_df['Survived']
X_test, y_test = raw_test_df.drop('Survived', axis=1), raw_test_df['Survived']

raw_train_df.sample(5, random_state=42)

# Prepare Pipeline

In [ ]:
%%run_and_save blueprints/pipeline.py


clip_outliers = [
    (
        'Float',
        tr.ClipOutliers(std_band=3),
        make_column_selector(dtype_include=['float64'])
    )
]

fillna = [
    (
        'Mean',
        s4p.SimpleImputer(strategy='mean'),
        make_column_selector(dtype_include=['float64'])
    ),
    (
        'Most Frequent',
        s4p.SimpleImputer(strategy='most_frequent'),
        make_column_selector(dtype_include=['int64', 'object'])
    )
]

standardize = [
    (
        'Floats Ints',
        s4p.StandardScaler(),
        ['Age', 'Fare']
    )
]


steps_main = [
    ('Format Cabins', tr.Cabin() ),
    ('Set working columns', tr.SetupFeatures(cols_ignore=['PassengerId', 'Name', 'Ticket', 'Parch']) ),
    ('Clip Outliers', s4p.ColumnTransformer(clip_outliers, remainder='passthrough') ),
    ('Standardize', s4p.ColumnTransformer(standardize, remainder='passthrough')),
    ('Fill NaN', s4p.ColumnTransformer(fillna, remainder='passthrough') ),
    ('Convert Types', tr.AsTypes(coltypes_overwrite={'Age': 'float64'}) ),
    ('One Hot', s4p.OneHotEncoder(cols_select=['Pclass', 'Sex', 'Cabin', 'SibSp', 'Embarked']) ),
    ('LogisticRegression', LogisticRegression(max_iter=200, random_state=42))
]

pipe = Pipeline(steps_main)

# Grid Params

In [ ]:
%%run_and_save blueprints/grid_params.py


ml_params_distributions = {
    'Clip Outliers__Float__std_band': [3.],
    'LogisticRegression__C': uniform(0.1, 3.),
}

rs = RandomizedSearchCV (
    pipe,
    param_distributions = ml_params_distributions,
    n_iter=1,
    n_jobs=-1,
    refit=True,
    cv=CVSplitter(5, 80),
    return_train_score=True
)

# ML flow run

In [ ]:
mlflow.sklearn.autolog(max_tuning_runs=3)

with mlflow.start_run(run_name='Try Standardized') as run:
    mlflow.set_tag()
    mlflow.log_artifact('blueprints/pipeline.py', 'blueprints')
    mlflow.log_artifact('blueprints/grid_params.py', 'blueprints')
    rs.fit(X_train, y_train)

# Refit best model

In [ ]:
url_local_model = 'file:///Users/i538262/Desktop/Developments/PERSO/KAGGLE%20Titanic/mlruns/0/fd8f4ddca0054f96ac4e7b1746396736/artifacts/best_estimator/model.pkl'
with urllib.request.urlopen(url_local_model) as model_file:
    best_model = pickle.load(model_file)
    
best_model_allfit = clone(best_model)
best_model_allfit.fit(X_all, y_all)

accuracy_score(y_all, best_model_allfit.predict(X_all))

# Submit models predictions

In [ ]:
submit('submissions/final_logsitic.csv', raw_sub_df.PassengerId, best_model_allfit.predict(raw_sub_df))

# Code tests

In [ ]:
dir(rep)

In [ ]:
rep.head.commit.message

In [ ]:
[c.message for c in list(rep.iter_commits('master', max_count=5))]

In [45]:
rep.active_branch.name

'master'